# Job Board Scraping Lab

In this lab you will first see a minimal but fully functional code snippet to scrape the LinkedIn Job Search webpage. You will then work on top of the example code and complete several chanllenges.

### Some Resources 

- [Requests library](http://docs.python-requests.org/en/master/#the-user-guide) documentation 
- [Beautiful Soup Doc](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Urllib](https://docs.python.org/3/library/urllib.html#module-urllib)
- [re lib](https://docs.python.org/3/library/re.html)
- [Scrapy](https://scrapy.org/)
- [List of HTTP status codes](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes)
- [HTML basics](http://www.simplehtmlguide.com/cheatsheet.php)
- [CSS basics](https://www.cssbasics.com/#page_start)

In [15]:
# Import the required libraries
import pandas as pd
from bs4 import BeautifulSoup
import requests

"""
This function searches job posts from LinkedIn and converts the results into a dataframe.
"""
def scrape_linkedin_job_search(keywords):
    
    # Define the base url to be scraped.
    # All uppercase variable name signifies this is a constant and its value should never unchange
    BASE_URL = 'https://www.linkedin.com/jobs/search/?'
        
    # Assemble the full url with parameters
    scrape_url = ''.join([BASE_URL, 'keywords=', keywords])

    # Create a request to get the data from the server 
    page = requests.get(scrape_url)
    soup = BeautifulSoup(page.text, 'html.parser')
    # Create an empty dataframe with the columns consisting of the information you want to capture
    columns = ['Title', 'Company', 'Location']
    data = pd.DataFrame(columns=columns)

    # Retrieve HTML code from the webpage. Parse the HTML into a list of "cards".
    # Then in each job card, extract the job title, company, and location data.
    titles = []
    companies = []
    locations = []
    for card in soup.select("div.result-card__contents"):
        title = card.findChild("h3", recursive=False)
        company = card.findChild("h4", recursive=False)
        location = card.findChild("span", attrs={"class": "job-result-card__location"}, recursive=True)
        titles.append(title.string)
        companies.append(company.string)
        locations.append(location.string)

    # Inject job titles, companies, and locations into the empty dataframe
    zipped = zip(titles, companies, locations)
    for z in list(zipped):
        data=data.append({'Title' : z[0] , 'Company' : z[1], 'Location': z[2]} , ignore_index=True)

    # Return dataframe
    return data

In [16]:
# Example to call the function

results = scrape_linkedin_job_search('data%20analysis')
results

,Title,Company,Location
0,IT Senior Associate,Kearney & Company,"McLean, VA, US"
1,Intern: Data Analysis/Process Improvement Prog...,Juniper Networks,"Sunnyvale, CA, US"
2,Associate / Business Analyst (Consulting Data ...,"Foresight Associates, LLC",Greater Minneapolis-St. Paul Area
3,"Software Engineer, Data Models & Analysis",Improbable,"Washington, D.C., DC, US"
4,"Manager, Reporting and Data Analysis",The Madison Square Garden Company,Greater New York City Area
5,Data Analysis Manager,Capital One,"McLean, VA, US"
6,Data Management and Analysis Services Analyst I,Conduent,"Ocoee, Florida"
7,"Manager, Data Analysis & Insights",The Coca-Cola Company,"Atlanta, GA, US"
8,Data Analyst,Viper Staffing Services L.L.C.,"Glendale, California, United States"
9,Sr. Bioinformatics Scientist,The EAC Agency,"Fremont, California, United States"


## Challenge 1

The first challenge for you is to update the `scrape_linkedin_job_search` function by adding a new parameter called `num_pages`. This will allow you to search more than 25 jobs with this function. Suggested steps:

1. Go to https://www.linkedin.com/jobs/search/?keywords=data%20analysis in your browser.
1. Scroll down the left panel and click the page 2 link. Look at how the URL changes and identify the page offset parameter.
1. Add `num_pages` as a new param to the `scrape_linkedin_job_search` function. Update the function code so that it uses a "for" loop to retrieve several pages of search results.
1. Test your new function by scraping 5 pages of the search results.

Hint: Prepare for the case where there are less than 5 pages of search results. Your function should be robust enough to **not** trigger errors. Simply skip making additional searches and return all results if the search already reaches the end.

In [17]:
# Import the required libraries
import pandas as pd
from bs4 import BeautifulSoup
import requests

"""
This function searches job posts from LinkedIn and converts the results into a dataframe.
"""
def scrape_linkedin_job_search2(keywords,numpages):
    
    # Define the base url to be scraped.
    # All uppercase variable name signifies this is a constant and its value should never unchange
    BASE_URL = 'https://www.linkedin.com/jobs/search/?'
    
    # Create an empty dataframe with the columns consisting of the information you want to capture
    columns = ['Title', 'Company', 'Location']
    data = pd.DataFrame(columns=columns)
    #https://www.linkedin.com/jobs/search/?geoId=101282230&keywords=data%20analysis&location=Germany&start=25
    for page in range(numpages):
        numpages_num=page*25
        # Assemble the full url with parameters
        scrape_url = ''.join([BASE_URL, 'keywords=', keywords,'&start=',str(numpages_num)])

        # Create a request to get the data from the server 
        page = requests.get(scrape_url)
        soup = BeautifulSoup(page.text, 'html.parser')

        # Retrieve HTML code from the webpage. Parse the HTML into a list of "cards".
        # Then in each job card, extract the job title, company, and location data.
        titles = []
        companies = []
        locations = []
        for card in soup.select("div.result-card__contents"):
            title = card.findChild("h3", recursive=False)
            company = card.findChild("h4", recursive=False)
            location = card.findChild("span", attrs={"class": "job-result-card__location"}, recursive=True)
            titles.append(title.string)
            companies.append(company.string)
            locations.append(location.string)

        # Inject job titles, companies, and locations into the empty dataframe
        zipped = zip(titles, companies, locations)
        for z in list(zipped):
            data=data.append({'Title' : z[0] , 'Company' : z[1], 'Location': z[2]} , ignore_index=True)

    # Return dataframe
    return data

In [22]:
# your code here
results2 = scrape_linkedin_job_search2('data%20analysis',5)
results2

,Title,Company,Location
0,Sr. Data Analyst – Predictive & Inferential An...,Peyton Resource Group,"Arlington, Texas"
1,"Manager, Reporting and Data Analysis",The Madison Square Garden Company,Greater New York City Area
2,Data Analysis Manager,Addison Group,"Houston, TX, US"
3,"Manager, Data and Analysis",Digitas North America,Greater New York City Area
4,Intern: Data Analysis/Process Improvement Prog...,Juniper Networks,"Sunnyvale, CA, US"
...,...,...,...
120,Data Analyst,24/7 Wall St.,Greater New York City Area
121,Senior Data Analyst,Trexin Consulting,"Chicago, Illinois"
122,Data Analyst,24 Seven LLC,"Portland, Oregon"
123,Data Analyst,"Partner's Consulting, Inc.","Philadelphia, Pennsylvania"


## Challenge 2

Further improve your function so that it can search jobs in a specific country. Add the 3rd param to your function called `country`. The steps are identical to those in Challange 1.

In [19]:
# your code here
# Import the required libraries
import pandas as pd
from bs4 import BeautifulSoup
import requests

"""
This function searches job posts from LinkedIn and converts the results into a dataframe.
"""
def scrape_linkedin_job_search3(keywords,numpages,country):
    
    # Define the base url to be scraped.
    # All uppercase variable name signifies this is a constant and its value should never unchange
    BASE_URL = 'https://www.linkedin.com/jobs/search/?'
    
    # Create an empty dataframe with the columns consisting of the information you want to capture
    columns = ['Title', 'Company', 'Location']
    data = pd.DataFrame(columns=columns)
    
    for x in range(numpages):
        numpages_num=x*25
        # Assemble the full url with parameters
        scrape_url = ''.join([BASE_URL, 'keywords=', keywords,'&start=',str(numpages_num),'&location=',country])

        # Create a request to get the data from the server 
        page = requests.get(scrape_url)
        soup = BeautifulSoup(page.text, 'html.parser')

        # Retrieve HTML code from the webpage. Parse the HTML into a list of "cards".
        # Then in each job card, extract the job title, company, and location data.
        titles = []
        companies = []
        locations = []
        for card in soup.select("div.result-card__contents"):
            title = card.findChild("h3", recursive=False)
            company = card.findChild("h4", recursive=False)
            location = card.findChild("span", attrs={"class": "job-result-card__location"}, recursive=True)
            titles.append(title.string)
            companies.append(company.string)
            locations.append(location.string)

        # Inject job titles, companies, and locations into the empty dataframe
        zipped = zip(titles, companies, locations)
        for z in list(zipped):
            data=data.append({'Title' : z[0] , 'Company' : z[1], 'Location': z[2]} , ignore_index=True)

    # Return dataframe
    return data

In [21]:
scrape_linkedin_job_search3('data%20analysis',2,'germany')

,Title,Company,Location
0,Senior Data Analyst,Takeaway.com,"Berlin, DE"
1,Head of Risk Analytics & Data Science (all gen...,Spotcap Global,"Berlin und Umgebung, Deutschland"
2,Senior Data Analyst,komoot,"Berlin, Berlin, Germany"
3,Senior Data Analyst,N26,"Berlin, DE"
4,Data Analyst,Amazon,"Munich, DE"
5,Data Analyst - Finance,Jimdo,"Hamburg, Hamburg, Germany"
6,Data Science Intern,AiSight,Berlin
7,Data Analyst,Euro London Appointments,"Berlin, Berlin, Germany"
8,Project Management Analyst (Intern),Adobe,"Hamburg, DE"
9,Senior Data Scientist,Glocomms,"Stuttgart Area, Germany"


## Challenge 3

Add the 4th param called `num_days` to your function to allow it to search jobs posted in the past X days. Note that in the LinkedIn job search the searched timespan is specified with the following param:

```
f_TPR=r259200
```

The number part in the param value is the number of seconds. 259,200 seconds equal to 3 days. You need to convert `num_days` to number of seconds and supply that info to LinkedIn job search.

In [27]:
# your code here
# your code here
# Import the required libraries
import pandas as pd
from bs4 import BeautifulSoup
import requests

"""
This function searches job posts from LinkedIn and converts the results into a dataframe.
"""
def scrape_linkedin_job_search4(keywords,numpages,country,num_days):
    
    # Define the base url to be scraped.
    # All uppercase variable name signifies this is a constant and its value should never unchange
    BASE_URL = 'https://www.linkedin.com/jobs/search/?'
    
    # Create an empty dataframe with the columns consisting of the information you want to capture
    columns = ['Title', 'Company', 'Location']
    data = pd.DataFrame(columns=columns)
    num_secs=num_days*3600
    for x in range(numpages):
        numpages_num=x*25
        # Assemble the full url with parameters
        scrape_url = ''.join([BASE_URL,'f_TPR=r',str(num_secs) ,'keywords=', keywords,'&start=',str(numpages_num),'&location=',country])

        # Create a request to get the data from the server 
        page = requests.get(scrape_url)
        soup = BeautifulSoup(page.text, 'html.parser')

        # Retrieve HTML code from the webpage. Parse the HTML into a list of "cards".
        # Then in each job card, extract the job title, company, and location data.
        titles = []
        companies = []
        locations = []
        for card in soup.select("div.result-card__contents"):
            title = card.findChild("h3", recursive=False)
            company = card.findChild("h4", recursive=False)
            location = card.findChild("span", attrs={"class": "job-result-card__location"}, recursive=True)
            titles.append(title.string)
            companies.append(company.string)
            locations.append(location.string)

        # Inject job titles, companies, and locations into the empty dataframe
        zipped = zip(titles, companies, locations)
        for z in list(zipped):
            data=data.append({'Title' : z[0] , 'Company' : z[1], 'Location': z[2]} , ignore_index=True)

    # Return dataframe
    return data

In [28]:
scrape_linkedin_job_search4('data%20analysis',2,'germany',7)

,Title,Company,Location
0,Maschineneinrichter (m/w/d) in der pharmazeuti...,Pfizer,"Freiburg, DE"
1,Technischer Einkäufer Laborautomation (m/w/d),Abbott,"Hamburg, DE"
2,Praktikant Human Resources Development - Autom...,BorgWarner Inc.,"Kirchheimbolanden, DE"
3,Data Manager (m/w/d),Bisnode,"Darmstadt, DE"
4,Duales Studium Wirtschaftsingenieurwesen (m/w/d),ANDRITZ,"Dachau, DE"
5,Wissenschaftliche*r Mitarbeiter*in für ein int...,Hochschule Ruhr West University of Applied Sci...,"Mülheim an der Ruhr, 45479, DE"
6,Business Intelligence Consultant,Klara Consulting Ltd,"Munich, Bavaria, Germany"
7,Manager IT Infrastruktur (m/w/d),HAUTAU GmbH,"Helpsen, Niedersachsen, Deutschland"
8,HR Working Student (m/f),relayr,"Berlin Area, Germany"
9,General Manager Ultrasound IT (Viewpoint) (m/w...,GE Healthcare,"Munich Area, Germany"


## Bonus Challenge

Allow your function to also retrieve the "Seniority Level" of each job searched. Note that the Seniority Level info is not in the initial search results. You need to make a separate search request for each job card based on the `currentJobId` value which you can extract from the job card HTML.

After you obtain the Seniority Level info, update the function and add it to a new column of the returned dataframe.

In [59]:
# your code here
# your code here
# Import the required libraries
import pandas as pd
from bs4 import BeautifulSoup
import requests

"""
This function searches job posts from LinkedIn and converts the results into a dataframe.
"""
def scrape_linkedin_job_search5(keywords,numpages,country,num_days):
    
    # Define the base url to be scraped.
    # All uppercase variable name signifies this is a constant and its value should never unchange
    BASE_URL = 'https://www.linkedin.com/jobs/search/?'
    
    # Create an empty dataframe with the columns consisting of the information you want to capture
    columns = ['Title', 'Company', 'Location']
    data = pd.DataFrame(columns=columns)
    num_secs=num_days*3600
    for x in range(numpages):
        numpages_num=x*25
        # Assemble the full url with parameters
        scrape_url = ''.join([BASE_URL,'f_TPR=r',str(num_secs) ,'keywords=', keywords,'&start=',str(numpages_num),'&location=',country])

        # Create a request to get the data from the server 
        page = requests.get(scrape_url)
        soup = BeautifulSoup(page.text, 'html.parser')
        
        # Retrieve HTML code from the webpage. Parse the HTML into a list of "cards".
        # Then in each job card, extract the job title, company, and location data.
        titles = []
        companies = []
        locations = []
        seniority_level=[]
        for card in soup.select("div.result-card__contents"):
            title = card.findChild("h3", recursive=False)
            company = card.findChild("h4", recursive=False)
            location = card.findChild("span", attrs={"class": "job-result-card__location"}, recursive=True)
            titles.append(title.string)
            companies.append(company.string)
            locations.append(location.string)
            #print(card.find("a").attrs['href'])
            #print(card.a)
        
        #print(soup.body.find("div",class_="jobs-search-two-pane__results display-flex"))
        #for link in soup.find("div",class_="job-card-search--two-pane  jobs-search-results__list--card--viewport-tracking-2 job-card-search job-card-search--column job-card-search ember-view"):
        #    print(link)
        #    if 'href' in link.attrs:
        #        print(link.attrs['href'])
        #    
        sen_level="none"
        #print(sen_level)
            
            
            #currentJobId=1510822801         
        seniority_level.append(sen_level)
        
        
        # Inject job titles, companies, and locations into the empty dataframe
        zipped = zip(titles, companies, locations,seniority_level)
        for z in list(zipped):
            data=data.append({'Title' : z[0] , 'Company' : z[1], 'Location': z[2], 'Seniority_Level': z[3]} , ignore_index=True)
    
    # Return dataframe
    return data

In [60]:
# your code here

scrape_linkedin_job_search5('data%20analysis',2,'germany',1)

#Mid-Senior level
#class="jobs-box__body js-formatted-exp-body"

#class="jobs-box__body js-formatted-exp-body"

None
None


,Title,Company,Location,Seniority_Level
0,Project Finance Account Manager,Medpace,"Munich, DE",none
1,Project Finance Account Manager,Medpace,"Munich, DE",none


In [107]:
range(2)

range(0, 2)